In [1]:
import pandas as pd
import numpy as np 
import sqlalchemy

In [3]:
old_book_users = """
select user_id, prod_group from sanction where prod_group in 
('4M','4M_0FEES','APNA','APNA_4M','APNA_5M','BIDSO_1M','BZTM_1M','FLPKRT_1M','NINJA_1M')
"""
f1, old_book_users_t, error_reason = data_fetch_v2(old_book_users,  host="13.126.97.63")

                                    user_id prod_group
0      5828b717-bbd1-46fd-b579-4bc21cd2a0d8         4M
1      41c99020-32a6-4911-afd3-313f08de9e03         4M
2      a832e010-c86c-434c-8f3f-9c3d8c843617         4M
3      1f2b7fe3-77e6-43e3-96aa-7cf85f7f838e         4M
4      aa2b5e49-e2ec-4aa4-a4a8-6d69d7bce840         4M
...                                     ...        ...
22995  19d5ead6-cb48-4fba-a40c-410eab0ac31b   NINJA_1M
22996  64e9b8b2-f30c-4076-9d92-92e0d84376a1  FLPKRT_1M
22997  420d4baf-466f-4337-8bd1-e7d404631e71   NINJA_1M
22998  a772d500-cf63-401e-9dd1-c43a2afb5370   NINJA_1M
22999  3b318962-75fb-4849-ac2c-7b79292501a9   NINJA_1M

[23000 rows x 2 columns]
                                    user_id prod_group
0      5828b717-bbd1-46fd-b579-4bc21cd2a0d8         4M
1      41c99020-32a6-4911-afd3-313f08de9e03         4M
2      a832e010-c86c-434c-8f3f-9c3d8c843617         4M
3      1f2b7fe3-77e6-43e3-96aa-7cf85f7f838e         4M
4      aa2b5e49-e2ec-4aa4-a4a8-6d69d7bc

In [ ]:
statement_history = f"""
select created_at, billing_cycle, amount, charge_type
 from statement_history where payment_status = 'Paid'  and charge_type in 
 ('bMonthly','cLatefee') and user_id in {tuple(old_book_users_t.user_id)}
"""
f1, statement_history_t, error_reason = data_fetch_v2(statement_history, host="13.126.97.63")

In [43]:
cycle = [
'202411',
'202409','202408','202407','202406','202405','202404','202403','202402','202401','202312','202311','202310','202309','202308',
'202307','202306','202305','202304','202303','202302','202301','202212','202211','202210','202209','202208','202207','202206',
'202205','202204','202203','202202','202201','202112','202111''202110','202109','202108','202107','202106','202105','202104',
'202103','202102','202101','202012','202011',
]

In [39]:
paid = statement_history_t.groupby('charge_type')['amount'].sum()


In [ ]:
def statement_h_fun(users_not_there,cycle):
    fee_paid  = f"""
    SELECT s.user_id,
    (case when MONTH(s.due_date)<10 then concat(YEAR(s.due_date),"0", MONTH(s.due_date)) ELSE concat(YEAR(s.due_date),MONTH(s.due_date))  END) AS billing_cycle,
    
    SUM((case when s.payment_status = "Paid" AND s.charge_type = "bMonthly"  then s.amount/1.18 ELSE 0 END)) AS monthly_fee_paid,
    SUM((case when s.payment_status = "Paid" AND s.charge_type = "cLatefee"  then s.amount/1.18 ELSE 0 END)) AS late_fee_paid
    FROM statement_history AS s
    WHERE s.charge_type <> 'aTranch' and user_id in {tuple(users_not_there.user_id)}
    and billing_cycle = {cycle}
    GROUP BY 1,2
    """
    parse_date = {}
    # otp_varified_user=pd.read_sql(otp_varified, db_connection)
    flag,fee_paid_data1, error_msg = utility.data_fetch_v2(fee_paid)
    # fee_paid_data.head()
    return fee_paid_data1

In [ ]:
fee_paid_data = pd.DataFrame({'user_id' : []})

In [ ]:

for i in range(len(billing_cycle_list)): 
    users_not_there = users_4m[~(users_4m['user_id'].isin(fee_paid_data.user_id))]
    print(len(users_not_there))
    fee_paid_chunk = statement_h_fun(users_not_there,billing_cycle_list[i])
    fee_paid_data = fee_paid_data.append(fee_paid_chunk)